In [80]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
import csv

### Primer acercamiento a Machine Learning (KNN)

In [90]:
dfTest = pd.read_csv('../data/TRAIN_TEST_corrected/test_corrected.csv')
dfTrain = pd.read_csv('../data/TRAIN_TEST_corrected/train_corrected.csv')
dfExternalCoords = pd.read_csv('../data/externalData/coordenadas_barrios.csv')

In [91]:
dfTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 11 columns):
id                     14166 non-null int64
year_created           14166 non-null int64
month_created          14166 non-null int64
day_created            14166 non-null int64
property_type          14166 non-null object
place_name             14166 non-null object
state_name             14166 non-null object
lat                    10487 non-null float64
lon                    10487 non-null float64
surface_total_in_m2    11853 non-null float64
rooms                  7501 non-null float64
dtypes: float64(4), int64(4), object(3)
memory usage: 1.2+ MB


In [92]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004797 entries, 0 to 1004796
Data columns (total 12 columns):
id                     1004797 non-null object
year_created           1004797 non-null int64
month_created          1004797 non-null int64
day_created            1004797 non-null int64
property_type          1004797 non-null object
place_name             1004797 non-null object
state_name             1004797 non-null object
lat                    743693 non-null float64
lon                    743682 non-null float64
surface_total_in_m2    1004797 non-null float64
rooms                  594683 non-null float64
price_aprox_usd        1004797 non-null float64
dtypes: float64(5), int64(3), object(4)
memory usage: 92.0+ MB


In [93]:
dfExternalCoords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 4 columns):
place                        438 non-null object
lat                          435 non-null float64
lon                          435 non-null float64
distance_to_obelisco_mtrs    435 non-null float64
dtypes: float64(3), object(1)
memory usage: 13.8+ KB


## ToDo:
+ Resolver NaN's en lat / lon con externalData.
+ Reemplazar place_name con distancia al obelisco.
+ Z-codes Standardization.
+ OneHotEncode property_type y state_name.
+ Make Submit & Comparison
+ Cross Validation K-Folds -> Make Submit & Comparison

___
### Corriendo el primer KNN

In [87]:
dfUtil = dfTrain.loc[:, ('place_name', 'year_created')]\
                .groupby(by='place_name')\
                .count().reset_index()\
                .sort_values(by='year_created', ascending=False)\
                .rename(columns={'year_created':'place_ocurrences'})
print 'Cantidad de barrios con menos de 100 publicaciones:', dfUtil.loc[dfUtil.place_ocurrences < 100, ('place_ocurrences')].sum()

Cantidad de barrios con menos de 100 publicaciones: 7957


In [88]:
# Selecciono property_type
prop_type_labelEncoder = preprocessing.LabelEncoder()
prop_type_labelEncoder.fit(np.array(list(dfTrain.property_type.unique()), dtype='object'))
trainPropTypeIntEncoded = prop_type_labelEncoder.transform(dfTrain.property_type)

enc = preprocessing.OneHotEncoder(sparse=False)
matrixTrainPropTypeEnc = enc.fit_transform(trainPropTypeIntEncoded.reshape(len(trainPropTypeIntEncoded), 1))
oneHotCodeCategories = pd.DataFrame(matrixTrainPropTypeEnc, columns=prop_type_labelEncoder.classes_)
oneHotCodeCategories.shape
pd.concat([dfTrain.reset_index(drop=True), oneHotCodeCategories.reset_index(drop=True)], axis=1).head().drop('property_type', axis=1)

,id,year_created,month_created,day_created,place_name,state_name,lat,lon,surface_total_in_m2,rooms,price_aprox_usd,PH,apartment,house,store
0,1771e4252915a911b8eb4964cde2790d5623bd33,2013,8,28,Caseros,Bs.As. G.B.A. Zona Oeste,-34.600116,-58.565334,62.0,3.0,130650.48,0.0,1.0,0.0,0.0
1,d63560d1f8f797a6d412814a27ec570beafb9d90,2013,8,28,Villa Crespo,Capital Federal,-34.597274,-58.439479,44.0,2.0,78500.00,0.0,1.0,0.0,0.0
2,486c44c6ae60cc40aa7aac085bf20a83a3e68e0e,2013,8,28,General Rodríguez,Bs.As. G.B.A. Zona Oeste,NaN,NaN,150.0,NaN,243081.79,0.0,0.0,1.0,0.0
3,3310de102f192c4a03a415e3f648799caef02da7,2013,8,28,Las Cañitas,Capital Federal,-34.567401,-58.434775,250.0,4.0,710000.00,0.0,1.0,0.0,0.0
4,005481f73ee4472d5c42f326537cc319af2a5f40,2013,8,28,Flores,Capital Federal,-34.626409,-58.459324,40.0,1.0,110714.41,0.0,1.0,0.0,0.0


In [89]:
dfTrain.shape

(1004797, 12)

In [ ]:
# Tengo que pasar property_type, place_name y state_name a datos numéricos:

#le = preprocessing.LabelEncoder()
#le.fit(np.array(list(set(list(dfTrain.property_type.unique()) + list(dfTest.property_type.unique()))), dtype='object'))
#dfTrain.property_type = le.transform(dfTrain.property_type)
#dfTest.property_type = le.transform(dfTest.property_type)

#le = preprocessing.LabelEncoder()
#le.fit(np.array(list(set(list(dfTrain.place_name.unique()) + list(dfTest.place_name.unique()))), dtype='object'))
#dfTrain.place_name = le.transform(dfTrain.place_name)
#dfTest.place_name = le.transform(dfTest.place_name)

#le = preprocessing.LabelEncoder()
#le.fit(np.array(list(set(list(dfTrain.state_name.unique()) + list(dfTest.state_name.unique()))), dtype='object'))
#dfTrain.state_name = le.transform(dfTrain.state_name)
#dfTest.state_name = le.transform(dfTest.state_name)

In [ ]:
# Lleno los valores faltantes (NaN's) con la clase predominante en esa columna

imp = preprocessing.Imputer(missing_values='NaN', strategy='most_frequent')
imp.fit(dfTrain[['rooms']])
dfTrain.rooms = imp.transform(dfTrain[['rooms']]).ravel()

imp = preprocessing.Imputer(missing_values='NaN', strategy='mean')
imp.fit(dfTrain[['lat']])
dfTrain.lat = imp.transform(dfTrain[['lat']]).ravel()
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean')
imp.fit(dfTrain[['lon']])
dfTrain.lon = imp.transform(dfTrain[['lon']]).ravel()

imp = preprocessing.Imputer(missing_values='NaN', strategy='most_frequent')
imp.fit(dfTest[['rooms']])
dfTest.rooms = imp.transform(dfTest[['rooms']]).ravel()

imp = preprocessing.Imputer(missing_values='NaN', strategy='mean')
imp.fit(dfTest[['lat']])
dfTest.lat = imp.transform(dfTest[['lat']]).ravel()
imp = preprocessing.Imputer(missing_values='NaN', strategy='mean')
imp.fit(dfTest[['lon']])
dfTest.lon = imp.transform(dfTest[['lon']]).ravel()

imp = preprocessing.Imputer(missing_values='NaN', strategy='mean')
imp.fit(dfTest[['surface_total_in_m2']])
dfTest.surface_total_in_m2 = imp.transform(dfTest[['surface_total_in_m2']]).ravel()

In [ ]:
dfTrain.info()

In [ ]:
dfTest.info()

In [ ]:
# Genero TRAIN y TARGET
target = dfTrain.price_usd
train = dfTrain.drop('price_usd', axis=1)

testIds = dfTest.loc[:, ('id')]
testVals = dfTest.drop('id', axis=1)

In [ ]:
knn = KNeighborsRegressor(n_neighbors=30, weights='distance', 
                          algorithm='kd_tree', leaf_size=30, 
                          metric='euclidean', n_jobs=-1)

knn.fit(train, target)

In [ ]:
predictions = knn.predict(testVals)

In [ ]:
# Escribo a archivo

with open('../data/predictions/2nd_acercamiento.csv', 'w') as outfile:
        rows = [["id", "price_usd"]]

        for idTest, prediction in zip(testIds, predictions):
            rows.append([idTest, prediction])

        out_csv = csv.writer(outfile)
        out_csv.writerows(rows)